In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using DrakeVisualizer
using Plots
using RigidBodyTreeInspector

using BilevelTrajOpt
gr(size=(500,500), html_output_format=:png);

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [40]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005;

In [41]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0.1, 0. , 0.]
N = 10
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
sim_data = get_sim_data(x0,env,Δt,true)

println("Explicit Contacts")
traj_exp = BilevelTrajOpt.simulate(x0,env,Δt,N,ctrl!,implicit_contact=false);

Explicit Contacts
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded


In [76]:
traj_imp = zeros(6,N-1)
traj_imp_ip = zeros(6,N-1)
for i=1:N-1
    q0 = traj_exp[1:sim_data.num_q,i]
    v0 = traj_exp[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,i]
    u0 = zeros(sim_data.num_v)
    qnext = traj_exp[1:sim_data.num_q,i+1]
    vnext = traj_exp[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,i+1]
    τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);
    traj_imp[:,i] = x_sol_auglag
    τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);
    traj_imp_ip[:,i] = x_sol_ip
end

Solve_Succeeded


Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded
Solve_Succeeded


In [77]:
j = 6
display(traj_exp[sim_data.num_q+sim_data.num_v+2:end,2:j+1])
display(traj_imp[:,1:j])
display(traj_imp_ip[:,1:j])

6×6 Array{Float64,2}:
 2.81235e-8  2.38686e-8  1.27964e-7  2.07837e-6  0.108827    0.111155  
 6.12471e-8  1.21273e-7  2.40798e-7  3.45272e-6  0.0928791   0.0944082 
 0.5         0.500017    0.49995     0.500031    0.147567    0.111155  
 6.12471e-8  1.21273e-7  2.40798e-7  3.45272e-6  0.0928791   0.0944082 
 0.075475    0.0509492   0.0264266   0.00190019  4.34442e-8  4.45357e-8
 1.0         1.00004     0.999902    1.00009     0.999973    1.0       

6×6 Array{Float64,2}:
 0.00481002  0.00483432  0.0048215   0.00482999  0.105632    0.125139  
 0.00962812  0.00959262  0.00961051  0.00960023  0.124991    0.124861  
 0.495182    0.495263    0.495163    0.495264    0.144372    0.12514   
 0.00962812  0.00959262  0.00961051  0.00960023  0.124991    0.124861  
 0.00693177  0.00693134  0.0069314   0.00693148  0.00693147  0.00693147
 1.0         1.00006     0.99991     1.00009     0.999973    1.0       

6×6 Array{Float64,2}:
 0.0       0.0        5.28871e-8  0.0         2.54314e-6  0.00999999
 0.0       0.0        1.30616e-7  0.0         9.06214e-6  0.00999999
 0.5       0.500017   0.49995     0.500029    0.038742    0.00999999
 0.0       0.0        1.30616e-7  0.0         9.06214e-6  0.00999999
 0.075475  0.0509492  0.0264267   0.00190017  0.0         0.00999999
 1.0       1.00004    0.999902    1.00009     0.999973    1.0       